In [2]:
# ===================================================================
# THE ONE, TRUE, FINAL TRAINING NOTEBOOK
# ===================================================================
import os, librosa, numpy as np, pandas as pd, joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dropout, Flatten, Dense
from tensorflow.keras.utils import to_categorical

print("--- Starting Final Keras Model Training ---")
RAVDESS_PATH = r'C:\Users\SWAROOPA\Desktop\jupyter projects\data\RAVDESS_PATH'

def extract_keras_features(file_path, max_pad_len=216):
    try:
        audio, sr = librosa.load(file_path, res_type='kaiser_fast', duration=3.0, sr=22050)
        mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
        pad_width = max_pad_len - mfccs.shape[1]
        mfccs = np.pad(mfccs, pad_width=((0,0),(0,pad_width)), mode='constant') if pad_width > 0 else mfccs[:,:max_pad_len]
        return mfccs.T # Transpose to (timesteps, features)
    except: return None

if not os.path.exists('saved_models'): os.makedirs('saved_models')

data = []
# (Same data processing loop as before)
senior_actors = [15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
emotion_dict = {'03': 'happy', '04': 'sad', '05': 'angry'}
for actor_dir in os.listdir(RAVDESS_PATH):
    actor_path = os.path.join(RAVDESS_PATH, actor_dir)
    if os.path.isdir(actor_path):
        for file in os.listdir(actor_path):
            if file.endswith('.wav'):
                try:
                    parts = file.split('-'); actor_num = int(parts[-1].split('.')[0])
                    gender = "male" if actor_num % 2 != 0 else "female"
                    age = "senior" if actor_num in senior_actors else "adult"
                    emotion = emotion_dict.get(parts[2])
                    if emotion:
                        feature = extract_keras_features(os.path.join(actor_path, file))
                        if feature is not None: data.append([feature, gender, age, emotion])
                except: continue
df = pd.DataFrame(data, columns=['feature', 'gender', 'age', 'emotion'])
print(f"Created dataset with {len(df)} samples.")

# --- GENDER MODEL ---
X = np.array(df['feature'].tolist())
y = np.array(df['gender'].tolist())
le_g = LabelEncoder(); y_g = le_g.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y_g, test_size=0.2, random_state=42, stratify=y_g)

# CORRECT SCALING
scaler_g = StandardScaler()
X_train_scaled = scaler_g.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test_scaled = scaler_g.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)
print("\n--- Training Gender Model (Keras CNN) ---")
model_g = Sequential([Conv1D(64, 5, padding='same', activation='relu', input_shape=X_train_scaled.shape[1:]), MaxPooling1D(4), Dropout(0.3), Flatten(), Dense(1, activation='sigmoid')])
model_g.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_g.fit(X_train_scaled, y_train, epochs=30, batch_size=32, validation_split=0.1, verbose=0)
loss, acc = model_g.evaluate(X_test_scaled, y_test, verbose=0); print(f"Gender Accuracy: {acc*100:.2f}%")
model_g.save('saved_models/gender_model.h5')
joblib.dump(le_g, 'saved_models/gender_label_encoder.pkl'); joblib.dump(scaler_g, 'saved_models/gender_scaler.pkl')

# --- AGE MODEL ---
df_male = df[df['gender'] == 'male']
X = np.array(df_male['feature'].tolist()); y = np.array(df_male['age'].tolist())
le_a = LabelEncoder(); y_a = to_categorical(le_a.fit_transform(y))
X_train, X_test, y_train, y_test = train_test_split(X, y_a, test_size=0.2, random_state=42, stratify=y_a)
scaler_a = StandardScaler()
X_train_scaled = scaler_a.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test_scaled = scaler_a.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)
print("\n--- Training Age Model (Keras CNN) ---")
model_a = Sequential([Conv1D(64, 5, padding='same', activation='relu', input_shape=X_train_scaled.shape[1:]), MaxPooling1D(4), Dropout(0.3), Flatten(), Dense(y_a.shape[1], activation='softmax')])
model_a.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_a.fit(X_train_scaled, y_train, epochs=30, batch_size=32, validation_split=0.1, verbose=0)
loss, acc = model_a.evaluate(X_test_scaled, y_test, verbose=0); print(f"Age Accuracy: {acc*100:.2f}%")
model_a.save('saved_models/age_model.h5')
joblib.dump(le_a, 'saved_models/age_label_encoder.pkl'); joblib.dump(scaler_a, 'saved_models/age_scaler.pkl')

# --- EMOTION MODEL ---
X = np.array(df['feature'].tolist()); y = np.array(df['emotion'].tolist())
le_e = LabelEncoder(); y_e = to_categorical(le_e.fit_transform(y))
X_train, X_test, y_train, y_test = train_test_split(X, y_e, test_size=0.2, random_state=42, stratify=y_e)
scaler_e = StandardScaler()
X_train_scaled = scaler_e.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test_scaled = scaler_e.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)
print("\n--- Training Emotion Model (Keras CNN) ---")
model_e = Sequential([Conv1D(64, 5, padding='same', activation='relu', input_shape=X_train_scaled.shape[1:]), MaxPooling1D(4), Dropout(0.3), Flatten(), Dense(y_e.shape[1], activation='softmax')])
model_e.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_e.fit(X_train_scaled, y_train, epochs=30, batch_size=32, validation_split=0.1, verbose=0)
loss, acc = model_e.evaluate(X_test_scaled, y_test, verbose=0); print(f"Emotion Accuracy: {acc*100:.2f}%")
model_e.save('saved_models/emotion_model.h5')
joblib.dump(le_e, 'saved_models/emotion_label_encoder.pkl'); joblib.dump(scaler_e, 'saved_models/emotion_scaler.pkl')

--- Starting Final Keras Model Training ---
Created dataset with 576 samples.

--- Training Gender Model (Keras CNN) ---


C:\Users\SWAROOPA\Desktop\jupyter projects\final_env\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Gender Accuracy: 97.41%

--- Training Age Model (Keras CNN) ---


C:\Users\SWAROOPA\Desktop\jupyter projects\final_env\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Age Accuracy: 91.38%

--- Training Emotion Model (Keras CNN) ---


C:\Users\SWAROOPA\Desktop\jupyter projects\final_env\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Emotion Accuracy: 85.34%


['saved_models/emotion_scaler.pkl']